In [1]:
#Transfer learning code with VGG16 network (Keras)
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.models import Input, Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
from glob import glob

Using TensorFlow backend.


In [3]:
#In case of loading full VGG16 model
model = VGG16()
model.summary()











_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         


In [4]:
#load, preprocess data and do predictions
images = glob('./random_images/*')
for i in images : 
    print('loading : {} image'.format(i))
    image = load_img(i, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = model.predict(image)
    label = decode_predictions(yhat)
    label = label[0][0]
    print('prediction : %s (%.2f%%)' % (label[1], label[2]*100))
    print('_________________________')

loading : ./random_images\burrito.jpg image
prediction : burrito (98.43%)
_________________________
loading : ./random_images\icecream.jpg image
prediction : ice_cream (95.84%)
_________________________
loading : ./random_images\jeep.jpg image
prediction : jeep (76.79%)
_________________________
loading : ./random_images\lemon.jpg image
prediction : lemon (54.00%)
_________________________
loading : ./random_images\pizza.jpg image
prediction : pizza (62.53%)
_________________________


In [5]:
#in case of loading VGG16 model without output layer and specify new input shape 
new_input = Input(shape=(640, 480, 3))
#model = VGG16(include_top=False, input_tensor=new_input, pooling='avg') with global averaging layer
trainable_model = VGG16(include_top=False, input_tensor=new_input) 
trainable_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 640, 480, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 640, 480, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 640, 480, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 320, 240, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 320, 240, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 320, 240, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 160, 120, 128)     0         
__________

In [34]:
# Freeze all the layers
for layer in trainable_model.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in trainable_model.layers:
    print(layer, layer.trainable)


<keras.engine.input_layer.InputLayer object at 0x000001CE76B1B2B0> False
<keras.layers.convolutional.Conv2D object at 0x000001CE76B1B6A0> False
<keras.layers.convolutional.Conv2D object at 0x000001CE75CF8128> False
<keras.layers.pooling.MaxPooling2D object at 0x000001CE76B1E390> False
<keras.layers.convolutional.Conv2D object at 0x000001CE76B346D8> False
<keras.layers.convolutional.Conv2D object at 0x000001CE76B36470> False
<keras.layers.pooling.MaxPooling2D object at 0x000001CE76B2E358> False
<keras.layers.convolutional.Conv2D object at 0x000001CE76B42860> False
<keras.layers.convolutional.Conv2D object at 0x000001CE76B47A20> False
<keras.layers.convolutional.Conv2D object at 0x000001CE76B4D908> False
<keras.layers.pooling.MaxPooling2D object at 0x000001CE7A049160> False
<keras.layers.convolutional.Conv2D object at 0x000001CE7A051780> False
<keras.layers.convolutional.Conv2D object at 0x000001CE7A05A940> False
<keras.layers.convolutional.Conv2D object at 0x000001CE7A05E828> False
<ker

In [35]:
custom_model = Sequential()
custom_model.add(trainable_model)
custom_model.add(Flatten())
custom_model.add(Dense(1024, activation='relu'))
custom_model.add(Dropout(0.5))
custom_model.add(Dense(3, activation='softmax'))
custom_model.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 20, 15, 512)       14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 153600)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              157287424 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 172,005,187
Trainable params: 157,290,499
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
print(label)

[[('n02814533', 'beach_wagon', 0.23981003), ('n03977966', 'police_van', 0.20628625), ('n03594945', 'jeep', 0.20500402), ('n03770679', 'minivan', 0.058818385), ('n03670208', 'limousine', 0.054763205)]]
